In [198]:
%%capture
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import statnlpbook.util as util
import statnlpbook.parsing as parsing
import statnlpbook.transition as transition

util.execute_notebook('Transition-based dependency parsing.ipynb')

<!---
Latex Macros
-->
$$
\newcommand{\Xs}{\mathcal{X}}
\newcommand{\Ys}{\mathcal{Y}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\balpha}{\boldsymbol{\alpha}}
\newcommand{\bbeta}{\boldsymbol{\beta}}
\newcommand{\aligns}{\mathbf{a}}
\newcommand{\align}{a}
\newcommand{\source}{\mathbf{s}}
\newcommand{\target}{\mathbf{t}}
\newcommand{\ssource}{s}
\newcommand{\starget}{t}
\newcommand{\repr}{\mathbf{f}}
\newcommand{\repry}{\mathbf{g}}
\newcommand{\x}{\mathbf{x}}
\newcommand{\prob}{p}
\newcommand{\a}{\alpha}
\newcommand{\b}{\beta}
\newcommand{\vocab}{V}
\newcommand{\params}{\boldsymbol{\theta}}
\newcommand{\param}{\theta}
\DeclareMathOperator{\perplexity}{PP}
\DeclareMathOperator{\argmax}{argmax}
\DeclareMathOperator{\argmin}{argmin}
\newcommand{\train}{\mathcal{D}}
\newcommand{\counts}[2]{\#_{#1}(#2) }
\newcommand{\length}[1]{\text{length}(#1) }
\newcommand{\indi}{\mathbb{I}}
$$

# Dependency Parsing

##  Motivation 

Constituent Parsers **more complex than needed**:
* Many applications only need grammatical **relations between words**
* Annotation costly and error prone
    * Should "Cadbury Dairy Milk" be a flat tree, or "(Cadbury (Dairy Milk))"?

**Dependency Parsing** addresses this...

[Stanford CoreNLP parser](http://nlp.stanford.edu:8080/corenlp/)

[spaCy](https://demos.explosion.ai/displacy/)

## Anatomy of a Dependency Tree

* Nodes:
    * Tokens of sentence
    * a ROOT node (akin to the S symbol in CFGs)
* Each **non-ROOT **token has **exactly one parent**
    * the word that controls its syntactic function, or
    * the word it depends on
* ROOT **has no parent**

### Example

In [22]:
tokens = ["ROOT", "Economic", "news", "had", "little", "effect", "on", "financial", "markets", "."]
arcs = set([(0,3, "root"), (0,9,"p"), (2,1,"amod"),(3,2,"nsubj"), (3, 5, "dobj"), (5,4,"amod"), (5,6, "prep"), (6,8,"pmod"), (8,7,"amod")])

transition.render_displacy(*transition.to_displacy_graph(arcs, tokens),"1000px")

## Dependency Parsing Approaches

### Graph-Based Parsing
* define $s_\params(\x,\y)$ over input sentences $\x$ and dependency graphs $\y$
* parsing: $\argmax_\y s_\params(\x,\y)$
* can often be framed as **finding maximum spanning trees** or other graph problems

### Transition-Based Parsing
* learn to perform the right action / transition in a bottom-up left right parser
* Train classifiers $s_\params(\x,y)$ where $y$ is an action, and $\x$ is solution built so far, and the remaining sentence


Currently the state-of-the-art...

## Parsing State 
Akin to bottom up parsing for CFGs...

A **buffer** 

In [157]:
transition.render_transitions_displacy(transitions[0:1], tokenized_sentence)

buffer,stack,parse,action
ROOT_0 Economic_1 news_2 had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,,,INIT


A word **stack** (only words, not trees as in CFG parsing)

In [182]:
transition.render_transitions_displacy(transitions[2:3], tokenized_sentence)

buffer,stack,parse,action
news_2 had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0 Economic_1,,shift


A current **parse tree**

In [209]:
transition.render_transitions_displacy(transitions[9:10], tokenized_sentence)

buffer,stack,parse,action
on_6 financial_7 markets_8 ._9,ROOT_0 had_3 effect_5,,rightArc-dobj


We use the following 
### Actions

**Shift**

push the word at the top of the buffer to the stack \\((S, i|B, A)\rightarrow(S|i, B, A)\\)

In [208]:
transition.render_transitions_displacy(transitions[0:2], tokenized_sentence)

buffer,stack,parse,action
ROOT_0 Economic_1 news_2 had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,,,INIT
Economic_1 news_2 had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0,,shift


**Reduce**

pop the word at the top of the stack if it has a head \\((S|i, B, A)\rightarrow(S, B, A)\\)

In [207]:
transition.render_transitions_displacy(transitions[13:15], tokenized_sentence)

buffer,stack,parse,action
._9,ROOT_0 had_3 effect_5 on_6 markets_8,,rightArc-pmod
._9,ROOT_0 had_3 effect_5 on_6,,reduce


**rightArc-[label]**

create a labeled arc from the word at the top of the stack \\(i\\) to the word at the top of the buffer \\(j\\) \\((S|i, j|B, A) \rightarrow (S|i|j, B, A\cup\{(i,j,l)\})\\)


In [205]:
transition.render_transitions_displacy(transitions[5:7], tokenized_sentence)

buffer,stack,parse,action
had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0,,leftArc-nsubj
little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0 had_3,,rightArc-root


**leftArc-[label]**

create a labeled arc from the word at the top of the buffer \\(j\\) to the word at the top of the stack \\(i\\) if \\(i\\) has no head \\((S|i, j|B, A) \rightarrow (S, j|B, A\cup\{(j,i,l)\})\\)


In [204]:
transition.render_transitions_displacy(transitions[2:4], tokenized_sentence)

buffer,stack,parse,action
news_2 had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0 Economic_1,,shift
news_2 had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0,,leftArc-amod


In [203]:
transition.render_transitions_displacy(transitions[:], tokenized_sentence)

buffer,stack,parse,action
ROOT_0 Economic_1 news_2 had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,,,INIT
Economic_1 news_2 had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0,,shift
news_2 had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0 Economic_1,,shift
news_2 had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0,,leftArc-amod
had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0 news_2,,shift
had_3 little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0,,leftArc-nsubj
little_4 effect_5 on_6 financial_7 markets_8 ._9,ROOT_0 had_3,,rightArc-root
effect_5 on_6 financial_7 markets_8 ._9,ROOT_0 had_3 little_4,,shift
effect_5 on_6 financial_7 markets_8 ._9,ROOT_0 had_3,,leftArc-amod
on_6 financial_7 markets_8 ._9,ROOT_0 had_3 effect_5,,rightArc-dobj


## Background Material

* [Mike Collins' PCFG lecture](http://www.cs.columbia.edu/~mcollins/courses/nlp2011/notes/pcfgs.pdf)
* Jurafsky & Martin, Chapter 12, Statistical Parsing